In [ ]:
import earthaccess
import ipywidgets as widgets
from ipywidgets import GridspecLayout
import pandas as pd
from datetime import datetime, timezone
import plotly.express as px
import itertools

from itables import show

from typing import Any, Dict, List, Optional, Type, Union

: 

In [ ]:
# f@#$QFac890fq#@
earthaccess.login(persist=True)

# TEMPO search - Datasets

The flow for working with this data seems to be as follows:
1. Use `search_datasets` to find relevant collections
2. Once you have one or more relevant collections, filter the granules within that collection (there may be thousands in each returned collection, you don't want all of them).
3. Use `search_data` with the `concept_id`s from the granules you want to get the download links.

The tricky part here is step 2. The user needs to select the collection they want, then you need to allow them some filtering. It mightmake sense to use [tabs, accordion or stack](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#accordion) in `ipywidgets` to better organize this (probably accordion or stack.) 

Display a table of collections, and have each row have a "select" button. Clicking that opens up a new section to filter the data. This will then populate that section with basic info on the data (number of granules, total size, etc.), as well as allow for filtering. Filtering will show the new reduced number of granules. 

In addition, they should be able to see basic data on each returned granule (size, time frame, spatial extent, etc.). Then a button allows them to click it and get the data link and/or download it directly.



## Quick test search and results

Get a bunch of results for a single project.

In [ ]:
tempo_datasets = earthaccess.search_datasets(
    project="TEMPO"
)

Summarize those results by showing basic info and a link to the results page.

In [ ]:
tempo_datasets.summary(show_links=True)

Grab a random dataset (search result) and show all the granules associated with it.

In [ ]:
granule_data = earthaccess.search_data(
    concept_id=tempo_datasets[0].concept_id()
)

In [ ]:
granule_data.summary()

In [20]:
# Sanity check, this will be a unit test later
assert len(granule_data.filter_temporal_extent()) == len(granule_data)

Show the spatial extent for one of those granules.

In [ ]:
granule_data[0].plot_spatial_extent()

## Temporal filtering

It looks like a lot of the granules scan the same locations, but presumably at different dates and times. So below I work on allowing filtering based on the date and time.

Show the different start and end dates (all combined into one) from all the returned data granules.

In [ ]:
granule_data.plot_temporal_extent()

Create a picker that allows you to choose an (optional) start and end date for the temporal extents, and filter to only granules within that range.

In [ ]:
# Create start and finish datetime pickers
start_datetime_picker = widgets.DatetimePicker(description='Start Datetime (UTC):',
                                               max=datetime.now(timezone.utc))

finish_datetime_picker = widgets.DatetimePicker(description='Finish Datetime (UTC):',
                                               max=datetime.now(timezone.utc))

# Create an output widget to display the result
output = widgets.Output()

# Function to update finish datetime picker minimum value based on start datetime picker value
def update_finish_min(change):
    finish_datetime_picker.min = start_datetime_picker.value

# Attach event handler to start datetime picker value change
start_datetime_picker.observe(update_finish_min, 'value')

# Function to handle button click event
def handle_button_click(button):
    with output:
        output.clear_output()
        
        filtered_granule_data = granule_data.filter_temporal_extent(start_datetime_picker.value, finish_datetime_picker.value)

        print(f"Number of granules within the specified temporal range: {len(filtered_granule_data)}")


# Create a button widget
button = widgets.Button(description='Filter dates')

# Attach event handler to button click
button.on_click(handle_button_click)

# Create a VBox to hold the widgets
app = widgets.VBox([start_datetime_picker, finish_datetime_picker, button, output])

# Display the app
app

## User spatial filtering

In [ ]:
from ipyleaflet import Map, DrawControl

# Create a map centered on North America
m = Map(center=[42, -90], zoom=3)

# Initialize the DrawControl
draw_control = DrawControl(
    rectangle={"shapeOptions": {"color": "#6bc2e5", "fillOpacity": 0.5}},
    polygon={},
    polyline={},
    circlemarker={},
    marker={},
    edit=False
)

# Function to handle the drawing event and extract coordinates
def handle_draw(target, action, geo_json):
    # Extract the geometry coordinates
    geometry_type = geo_json['geometry']['type']
    coordinates = geo_json['geometry']['coordinates']

    if geometry_type == 'Polygon':
        formatted_coords = coordinates[0]
        formatted_coords = [(coord[0], coord[1]) for coord in formatted_coords]
        filtered_granules = granule_data.filter_spatial_extent(formatted_coords)
        print(f"Filtered granules count: {len(filtered_granules)}")

# Attach the handler to the draw control
draw_control.on_draw(handle_draw)

# Add the draw control to the map
m.add_control(draw_control)

# Display the map
m


## Search box
- Project name (initially the only option available)
- Date range (via a date picker)
- Location range (via an interactive map, ideally)
- Instrument (autofills after selecting a project and extracting the instruments)

In [25]:
class SearchWidget:
    def __init__(self):
        # Display the whole description, don't truncate it
        self.style = {'description_width': 'initial'}

        self.params = dict()
        self.results = None
        self.widgets = dict()

        self.widgets['project'] = self._project_widget()
        self.widgets['start_date'] = self._start_date_widget()
        self.widgets['end_date'] = self._end_date_widget()
        self.widgets['instrument'] = self._instrument_widget()
        self.widgets['output'] = widgets.Output()

        self._grid = GridspecLayout(5, 2)
        self._grid[0, 0] = self.widgets['project']
        self._grid[3, 0] = self._search_button()
        self._grid[3, 1] = self._reset_button()
        self._grid[4, :] = self._output_widget()

    def __repr__(self) -> str:
        return "Run in a Jupyter Notebook to see the widget."
    
    def _ipython_display_(self):
        # Display the widget
        display(self._grid)

    def _output_widget(self) -> widgets.Output:
        output_widget = widgets.Output()
        return output_widget

    def _project_widget(self) -> widgets.Text:
        project_widget = widgets.Text(
            description="Project/Campaign:",
            style=self.style,
            disabled=self.results is not None,
        )
        return project_widget
    
    def _start_date_widget(self) -> widgets.DatePicker:
        start_date_widget = widgets.DatePicker(
            description="Start Date:",
            style=self.style,
        )
        return start_date_widget
    
    def _end_date_widget(self) -> widgets.DatePicker:
        end_date_widget = widgets.DatePicker(
            description="Start Date:",
            style=self.style,
        )
        return end_date_widget
    
    def _instrument_widget(self) -> widgets.Dropdown:
        instrument_widget = widgets.Dropdown(
            options=[],
            description="Instrument:",
            style=self.style,
        )
        return instrument_widget
    
    def _search_button(self) -> widgets.Button:
        search_widget = widgets.Button(
            description="Search",
            button_style="primary",
        )
        search_widget.on_click(self.on_search)
        return search_widget
    
    def _reset_button(self) -> widgets.Button:
        reset_widget = widgets.Button(
            description="Reset",
            button_style="warning",
        )
        reset_widget.on_click(self.on_reset)
        return reset_widget

    def on_search(self, button):
        # Assign parameter values
        self.params['project'] = self.widgets['project'].value
        self.params['start_date'] = self.widgets['start_date'].value
        self.params['end_date'] = self.widgets['end_date'].value
        self.params['instrument'] = self.widgets['instrument'].value

        print(f"Project: {self.widgets['project'].value}")
        print(f"Start date: {self.widgets['start_date'].value}")
        print(f"End date: {self.widgets['end_date'].value}")
        print(f"Instrument: {self.widgets['instrument'].value}")

        # Execute the search
        if self.results is None:
            self.results = earthaccess.search_datasets(
                project=self.params['project']
            )
        else:
            self.results = earthaccess.search_datasets(
                project=self.params['project'],
                start_date=self.params['start_date'],
                end_date=self.params['end_date'],
                instrument=self.params['instrument']
            )

        if len(self.results) > 0:
            self.display_results(self.results)

    def on_reset(self, button):
        # Reset widgets to initial values
        self.widgets['project'].value = self.widgets['project'].placeholder
        # self.widgets['start_date'].value = self.widgets['start_date'].placeholder
        # self.widgets['end_date'].value = self.widgets['end_date'].placeholder
        # self.widgets['instrument'].value = self.widgets['instrument'].placeholder

        # Reset parameters to empty
        self.params['project'] = None
        self.params['start_date'] = None
        self.params['end_date'] = None
        self.params['instrument'] = None
        
        # Re-enable the project widget and set focus to it
        self.widgets['project'].disabled = False
        self.widgets['project'].focus()

    def display_results(self, results):
        with self.widgets['output']:
            display(results)

In [ ]:
grid = SearchWidget()

grid